### Анализ случаев мошенничества в страховании 
- Начать с разведочного (первичного) анализа данных. Визуализируйте датасеты. Поищите явные “более мошеннические” сегменты.
- Построить различные модели, которые будут предсказывать вероятность мошеннического случая (переменную `bad`) на различных классификаторах
- Сравнить результаты градиентного бустинга деревьев с некоторыми параметрами (обсудим в следующий раз как их выбирать) и логистической регрессией (только не забудьте данные нормировать или применять, например, `QuantileTransformer`).
- Попробовать разные способы обработки категориальных признаков (заодно можете сравнить `lightgbm` и `catboost`)
- Объяснить важности признаков. Рекомендую библиотеку http://eli5.readthedocs.io/en/latest/
- Попробовать описать сегменты, где же мошенничают.
- Подумать над вопросом, как лучше свой результат донести менеджеру банкира?

#### Разведочный (первичный) анализ данных.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
plt.style.use('bmh')

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 400)

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
file = pd.read_csv("Data/Data_AlfaInsurance_170918.csv", sep=';', decimal=",")
train, test = train_test_split(file, test_size=0.2)
train.describe()

,Period_EvCl,Period_StEv,Policy_agent_cat,FLAG_Owner_bl,FLAG_Insurer_bl,Policy_KBM,Policy_KS,Policy_KT,Policy_KVS,FLAG_Policy_KO,FLAG_Policy_KP,FLAG_Policy_KPR,FLAG_Policy_type,VEH_age,VEH_capacity_type,FLAG_bad_region,FLAG_dsago,FLAG_prolong,Policy_loss_count,Damage_count,bad,Claim_type,FLAG_trial
count,6815.000000,6815.000000,6727.000000,6815.000000,6815.00000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6814.000000,6814.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000,6815.000000
mean,21.664857,144.956420,0.080476,0.028026,0.02920,0.913133,0.975378,1.366787,1.029552,0.250183,0.996772,0.013646,0.207483,9.008805,4.605958,0.324872,0.022010,0.203962,0.159061,1.101541,0.088628,0.310932,0.129567
std,57.372659,114.846387,0.063684,0.165060,0.16838,0.173738,0.099801,0.460251,0.142994,0.433150,0.056729,0.116026,0.405534,6.714267,1.323998,0.468361,0.146728,0.402971,0.538178,0.338277,0.284227,0.670365,0.335851
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.500000,0.500000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,4.000000,37.000000,0.055636,0.000000,0.00000,0.900000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,9.000000,126.000000,0.066844,0.000000,0.00000,0.950000,1.000000,1.500000,1.000000,0.000000,1.000000,0.000000,0.000000,8.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,19.000000,238.000000,0.097161,0.000000,0.00000,1.000000,1.000000,1.700000,1.000000,1.000000,1.000000,0.000000,0.000000,13.000000,6.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1673.000000,366.000000,0.962270,1.000000,1.00000,2.450000,1.000000,2.100000,1.800000,1.000000,1.000000,1.000000,1.000000,44.000000,7.000000,1.000000,1.000000,1.000000,9.000000,5.000000,1.000000,2.000000,1.000000


In [3]:
test_true = test.copy()
test = test.drop('bad', axis=1)
test.describe()

,Period_EvCl,Period_StEv,Policy_agent_cat,FLAG_Owner_bl,FLAG_Insurer_bl,Policy_KBM,Policy_KS,Policy_KT,Policy_KVS,FLAG_Policy_KO,FLAG_Policy_KP,FLAG_Policy_KPR,FLAG_Policy_type,VEH_age,VEH_capacity_type,FLAG_bad_region,FLAG_dsago,FLAG_prolong,Policy_loss_count,Damage_count,Claim_type,FLAG_trial
count,1704.000000,1704.000000,1686.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000
mean,20.965376,145.746479,0.081693,0.019366,0.019366,0.910006,0.976702,1.366461,1.034859,0.245892,0.997653,0.014085,0.208920,8.901995,4.592136,0.338028,0.016432,0.198357,0.150235,1.109742,0.346244,0.143192
std,48.021947,115.965027,0.071657,0.137849,0.137849,0.165945,0.095970,0.454743,0.153582,0.430741,0.048407,0.117874,0.406657,6.449434,1.319161,0.473177,0.127167,0.398879,0.540514,0.371060,0.707196,0.350372
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,5.000000,36.000000,0.056197,0.000000,0.000000,0.900000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,10.000000,131.000000,0.065974,0.000000,0.000000,0.950000,1.000000,1.500000,1.000000,0.000000,1.000000,0.000000,0.000000,8.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,19.000000,243.000000,0.097897,0.000000,0.000000,1.000000,1.000000,1.700000,1.000000,0.000000,1.000000,0.000000,0.000000,13.000000,6.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,592.000000,366.000000,1.274395,1.000000,1.000000,2.450000,1.000000,2.100000,1.800000,1.000000,1.000000,1.000000,1.000000,38.000000,7.000000,1.000000,1.000000,1.000000,11.000000,5.000000,2.000000,1.000000


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6815 entries, 4983 to 3783
Data columns (total 32 columns):
claim_id             6815 non-null object
Event_type           6815 non-null object
Period_EvCl          6815 non-null int64
Period_StEv          6815 non-null int64
Policy_agent_cat     6727 non-null float64
Owner_type           6815 non-null object
FLAG_Owner_bl        6815 non-null int64
Insurer_type         6815 non-null object
FLAG_Insurer_bl      6815 non-null int64
Policy_KBM           6815 non-null float64
Policy_KS            6815 non-null float64
Policy_KT            6815 non-null float64
Policy_KVS           6815 non-null float64
FLAG_Policy_KO       6815 non-null int64
FLAG_Policy_KP       6815 non-null int64
FLAG_Policy_KPR      6815 non-null int64
FLAG_Policy_type     6815 non-null int64
VEH_age              6814 non-null float64
VEH_aim_use          6815 non-null object
VEH_capacity_type    6814 non-null float64
VEH_model            6815 non-null object
VEH_type_

Отобразим количество пропущеной информации

In [5]:
total = file.isnull().sum()
null_rows = file.isnull().any(axis = 1)
total_bad = file[file.bad == 1].isnull().sum()
missing_data = pd.concat([total, total_bad], axis=1, keys=['Total', 'Total_bad'])
mis_col = missing_data[missing_data.Total > 0]['Total'].sort_values(ascending=False)
print("Максимум пустых значений в столбце: ",total.max())
print("Количество записей с пустыми ячейками: ",null_rows.sum())
print("Список столбцов, в которых присутствуют пустые ячейки\n", mis_col)

Максимум пустых значений в столбце:  106
Количество записей с пустыми ячейками:  119
Список столбцов, в которых присутствуют пустые ячейки
 Policy_agent_cat     106
Owner_region          11
VEH_type_name          1
VEH_capacity_type      1
VEH_age                1
Name: Total, dtype: int64


Надо что-то делать с пропущеной инфой, решаю удалить записи, в которых присутствует NaN, кроме параметра Policy_agent_cat

In [6]:
policy_mean = file.Policy_agent_cat.mean()
policy_mean

0.0807199103113398

## Напишем обработчик входящих данных

In [7]:
def munge_data(data):
    # Policy_agent_cat- замена пропусков на медиану
    
    data["Policy_agent_cat"] = data.apply(lambda r: policy_mean if pd.isnull(r["Policy_agent_cat"]) else r["Policy_agent_cat"], axis=1)
    
    # удаление записей с пропусками
    
    data = data.drop(data[data.isnull().any(axis = 1) == True].index)
    data = data.drop('claim_id', axis = 1)
    #determine the unique Event_type
    Event_types = data.Event_type.unique()
    #make a column Event_flag
    data['Event_flag'] =  np.int64(Event_types[0] == data.Event_type)
    
    Own_types = {'Юридическое Лицо': 1, 'Физическое Лицо':2,'ПБОЮЛ':3}
    #make a column Event_flag
    data['Owner_type_int'] =   data['Owner_type'].apply(lambda c: Own_types.get(c, -1))
    data['Insurer_type_int'] =   data['Insurer_type'].apply(lambda c: Own_types.get(c, -1))

    
    return data

In [8]:
file = munge_data(file)
test_munged = munge_data(test)
train_munged = munge_data(train)

In [9]:
f_num = file.select_dtypes(include = ['float64', 'int64'])
f_num = f_num.drop('bad', axis=1)
f_num.columns

Index(['Period_EvCl', 'Period_StEv', 'Policy_agent_cat', 'FLAG_Owner_bl', 'FLAG_Insurer_bl', 'Policy_KBM', 'Policy_KS', 'Policy_KT', 'Policy_KVS', 'FLAG_Policy_KO', 'FLAG_Policy_KP',
       'FLAG_Policy_KPR', 'FLAG_Policy_type', 'VEH_age', 'VEH_capacity_type', 'FLAG_bad_region', 'FLAG_dsago', 'FLAG_prolong', 'Policy_loss_count', 'Damage_count', 'Claim_type', 'FLAG_trial',
       'Event_flag', 'Owner_type_int', 'Insurer_type_int'],
      dtype='object')

In [10]:
scaler = StandardScaler()
scaler.fit(file[f_num.columns])

test_data_scaled = scaler.transform(test_munged[f_num.columns])
train_data_scaled = scaler.transform(train_munged[f_num.columns])
print(train_data_scaled.shape)
print(test_data_scaled.shape)

(6807, 25)
(1699, 25)


In [11]:
cv = StratifiedKFold(train_munged["bad"], n_folds=3, shuffle=True, random_state=1)
cv

sklearn.cross_validation.StratifiedKFold(labels=[1 0 0 ..., 0 0 0], n_folds=3, shuffle=True, random_state=1)

In [12]:
alg_frst = RandomForestClassifier(random_state=1, n_estimators=500, min_samples_split=8, min_samples_leaf=2)
scores = cross_val_score(alg_frst, train_data_scaled, train_munged["bad"], cv=cv, n_jobs=-1)
print("Accuracy (random forest): {}/{}".format(scores.mean(), scores.std()))


Accuracy (random forest): 0.9114147201410313/0.0007196972947798278


In [15]:
test = test.drop(test[test.isnull().any(axis = 1) == True].index)
test.describe()
test_true.describe()

,Period_EvCl,Period_StEv,Policy_agent_cat,FLAG_Owner_bl,FLAG_Insurer_bl,Policy_KBM,Policy_KS,Policy_KT,Policy_KVS,FLAG_Policy_KO,FLAG_Policy_KP,FLAG_Policy_KPR,FLAG_Policy_type,VEH_age,VEH_capacity_type,FLAG_bad_region,FLAG_dsago,FLAG_prolong,Policy_loss_count,Damage_count,bad,Claim_type,FLAG_trial
count,1704.000000,1704.000000,1686.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000,1704.000000
mean,20.965376,145.746479,0.081693,0.019366,0.019366,0.910006,0.976702,1.366461,1.034859,0.245892,0.997653,0.014085,0.208920,8.901995,4.592136,0.338028,0.016432,0.198357,0.150235,1.109742,0.083920,0.346244,0.143192
std,48.021947,115.965027,0.071657,0.137849,0.137849,0.165945,0.095970,0.454743,0.153582,0.430741,0.048407,0.117874,0.406657,6.449434,1.319161,0.473177,0.127167,0.398879,0.540514,0.371060,0.277349,0.707196,0.350372
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,5.000000,36.000000,0.056197,0.000000,0.000000,0.900000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,10.000000,131.000000,0.065974,0.000000,0.000000,0.950000,1.000000,1.500000,1.000000,0.000000,1.000000,0.000000,0.000000,8.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,19.000000,243.000000,0.097897,0.000000,0.000000,1.000000,1.000000,1.700000,1.000000,0.000000,1.000000,0.000000,0.000000,13.000000,6.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,592.000000,366.000000,1.274395,1.000000,1.000000,2.450000,1.000000,2.100000,1.800000,1.000000,1.000000,1.000000,1.000000,38.000000,7.000000,1.000000,1.000000,1.000000,11.000000,5.000000,1.000000,2.000000,1.000000


In [16]:
test_temp = test_true.copy()

In [ ]:
test = test.drop(test[test.isnull().any(axis = 1) == True].index)
test.describe()

In [14]:
predictions.shape

NameError: name 'predictions' is not defined

In [ ]:
alg_test = alg_frst

alg_test.fit(train_data_scaled, train_munged["bad"])

predictions = alg_test.predict(test_data_scaled)
predictions.shape
submission = pd.DataFrame({
    "bad_true": test_true["bad"],
    "bad": predictions
})
submission

In [ ]:
bad = pd.crosstab(file.Owner_region, file.bad)[1]
all = pd.crosstab(file.Owner_region, file.bad).sum(axis=1)
print(pd.crosstab(file.Owner_region, file.bad)[(bad/all)==0].sort_values(by=0, ascending=False))
(bad/all).sort_values(ascending=False).plot(kind="bar", figsize=(10, 10), color='r')

In [ ]:
pd.crosstab(file.Owner_region, file.bad).sort_values(by= 0,ascending=False).plot(kind="bar", stacked=True, figsize=(10, 10))

In [ ]:
klas = file.select_dtypes(include = ['object'])
for kl in klas:
    print(kl, ' ',klas[kl].unique())

In [ ]:
f_num.hist(figsize=(16, 20), bins=30, xlabelsize=8, ylabelsize=8)

In [ ]:
#scatter plot grlivarea/saleprice
var = 'Period_EvCl'
data = pd.concat([file['bad'], file[var]], axis=1)
data.plot.scatter(x='bad', y=var , alpha = 0.2);

In [ ]:
corrmat = file.corr()
f, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corrmat, vmax=1, vmin=-1, square=True);

In [ ]:
x_var = 'Event_flag'
y_var = 'Damage_count'
data = pd.concat([file[x_var], file[y_var]], axis=1)
data.plot.scatter(x=y_var, y=x_var);

In [ ]:
sns.pairplot(file, vars=["FLAG_trial", "Event_flag", "Policy_loss_count", "FLAG_bad_region"], hue="bad", dropna=True)
sns.plt.show()

In [ ]:
predictors = f_num.columns.drop('bad')
print(predictors)

In [ ]:

selector = SelectKBest(f_classif, k=5)
selector.fit(train_munged[f_num.columns], train_munged["bad"])

scores = -np.log10(selector.pvalues_)

plt.bar(range(len(f_num.columns)), scores)
plt.xticks(range(len(f_num.columns)), f_num.columns, rotation='vertical')
plt.show()